In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input 
from tensorflow.keras.regularizers import l1_12

import random
import os

import nbimporter 
import useful 

def all_models(df_brut, k_iqr, column_name, column, train_coeff, val_coeff, test_coeff, lookback, units, l1, l2, learning_rate, epochs, batch_size, n_estimators, random_state) :
    """
    Args:
    df _brut (pd.DataFrame): le dataset initial 
    k (int): le coeff dans IQR
    column name (str): le nom de la colonne qui nous interesse pour la time series (ex: bid ou ask) 
    column (str): ask or bid
    """

    ####################### manage data
    data_brut = df_brut[column_name]
    # créer une nouvelle colonne qui aura les indices du dataframe original (le df_brut), pour pouvoir les garder
    df_brut['index_column'] = df_brut.index

    ####################### IQR
    # détecter les outliers avec IQR
    detected_outlier_indices = useful.detect_IQR(df_brut, k_iqr, column_name)

    # créer des labels et ecrire la liste des true outliers
    labels_first = df_brut['true_outliers']
    true_outlier_indices = df_brut[labels_first == 1].index.tolist()

    # la liste commune entre les outliers detectes avec IQR et les vrais outliers
    comm = [i for i in detected_outlier_indices.tolist() if i in true_outlier_indices]

    # affichage et visualisation
    print("Metriques IQR")
    useful.afficher_metriques(comm, true_outlier_indices, detected_outlier_indices.tolist(), column)

    ####################### update data after finding first big outliers with IQR
    # enlever les outliers deja detectés par le IQR et crer un nouveau dataframe df sans ces outliers aberrants
    df = df_brut.drop(detected_outlier_indices)
    df = df.reset_index(drop=True)

    # créer les nouveaux labels et les nouveaux indices des vrais outliers
    labels = df['true_outliers']
    outlier_indices = df[labels == 1].index.tolist()
    data = df[column_name]

    ####################### LSTM + RF
    # créer les séquences pour le LSTM
    X,y,y_labels = useful.create_sequences(data, labels, lookback)

    # découper les données en train (60%), validation (20%) et test (20%)
    # la taille du debut à la fin de train (donc le nombre de données de train)
    train_size = int(len(X) * train_coeff)
    # la taille des données de validation
    val_size = int(len(X) * val_coeff)

    X_train, y_train, y_train_labels = X[:train_size], y[:train_size], y_labels[:train_size]
    X_val, y_val, y_val_labels = X[train_size:train_size+val_size], y[train_size:train_size+val_size], y_labels[train_size:train_size+val_size]
    X_test, y_test, y_test_labels = X[train_size+val_size:], y[train_size+val_size:], y_labels[train_size+val_size:]

    # reshape pour LSTM
    X_train_lstm = X_train.reshape(len(X_train), lookback, 1)
    X_val_lstm = X_val.reshape(len(X_val), lookback, 1)
    X_test_lstm = X_test.reshape(len(X_test), lookback, 1)

    # construire et entrainer le modèle LSTM
    model = useful.build_model(lookback, units, learning_rate, l1, l2)
    history = model.fit(X_train_lstm, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val_lstm, y_val), verbose=0)

    # affichage de la loss en fonction des epochs
    useful.plot_loss_epochs(history)

    # predire sur train, val et test
    y_train_pred = model.predict(X_train_lstm)
    y_val_pred = model.predict(X_val_lstm)
    y_test_pred = model.predict(X_test_lstm)

    # calculer les résidus
    train_residuals = y_train - y_train_pred.flatten()
    val_residuals = y_val - y_val_pred.flatten()
    test_residuals = y_test - y_test_pred.flatten()

    # préparer les données pour le classifier
    X_train_rf, y_train_rf = useful.prepade_classifier_data(X_train, train_residuals, y_train_pred, y_train, y_train_labels)
    X_val_rf, y_val_rf = useful.prepade_classifier_data(X_val, val_residuals, y_val_pred, y_val, y_val_labels)
    X_test_rf, y_test_rf = useful.prepade_classifier_data(X_test, test_residuals, y_test_pred, y_test, y_test_labels)

    # concatener les données de train et val pour le classifier
    X_rf_train = np.vstack((X_train_rf, X_val_rf))
    y_rf_train = np.hstack((y_train_rf, y_val_rf))

    # entrainer le classifier
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf_classifier.fit(X_rf_train, y_rf_train)

    # predire sur test
    y_rf_pred = rf_classifier.predict(X_test_rf)

    # les indices des outliers detectes par le Random Forest
    outlier_indices_rf = np.where(y_rf_pred == 1)[0]

    ####################### sauvegarde des modeles
    # sauvegarder le modèle LSTM et le classifier
    model.save('model_lstm.h5')
    dump(rf_classifier, 'rf_classifier.joblib')

    # affichage et visualisation
    print("Metriques LSTM + RF")
    # evaluation du classifier
    print("Classification report:")
    print(classification_report(y_test_rf, y_rf_pred))
    print("Confusion matrix:")
    print(confusion_matrix(y_test_rf, y_rf_pred))

    ####################### acces aux outliers detectes
    # calcul de l'indice de départ dans les données df
    test_start_index = train_size + val_size + lookback

    # indices des données du jeu de test dans les données df
    data_indices_in_test_set = np.arange(len(y_test)) + test_start_index

    # indices des outliers detectes dans les donnees df
    outlier_data_indices = data_indices_in_test_set[outlier_indices_rf]

    ####################### métriques totales avec indices originaux
    outliers_data_indices_df_brut_rf = df['index_column'][outlier_data_indices].tolist()
    outliers_data_indices_df_brut_rf.sort()

    # couper les listes au niveau de la partie "test" pour pouvoir faire les comparaisons sur cette partie
    nb_couper_test = int(len(df_brut) * (train_coeff + val_coeff))

    detected_outlier_indices_iqr_test = useful.couper_liste(detected_outlier_indices.tolist(), nb_couper_test)
    true_outlier_indices_test = useful.couper_liste(true_outlier_indices, nb_couper_test)

    detected_outliers_total = detected_outlier_indices_iqr_test + outliers_data_indices_df_brut_rf
    detected_outliers_total.sort()

    comm_test_total = [i for i in detected_outliers_total if i in true_outlier_indices_test]

    # affichage et visualisation
    print("Metriques totales")
    useful.afficher_metriques(comm_test_total, true_outlier_indices_test, detected_outliers_total, column)

    return df